In [1]:
# YOLOv8 Playing Card Detection Training
# Run this in Google Colab

# Install dependencies
!pip install ultralytics roboflow onnx tf2onnx

# Download dataset
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="behE2IvztJ8TEE7WISvo")
project = rf.workspace("poker-eqo1i").project("playing-card-dedbm")
version = project.version(1)
dataset = version.download("yolov8")

# Train YOLOv8 model
from ultralytics import YOLO

# Load a larger pretrained model for better accuracy
# Options: yolov8n.pt (fastest), yolov8s.pt (balanced), yolov8m.pt (accurate), yolov8l.pt (very accurate)
model = YOLO('yolov8s.pt')  # Small - better accuracy than nano, still fast

# Train the model with improved settings
results = model.train(
    data=f'{dataset.location}/data.yaml',
    epochs=200,  # More epochs for better learning
    imgsz=640,
    batch=16,
    name='card_detector_v2',
    patience=30,  # More patience before early stopping
    save=True,
    plots=True,
    # Data augmentation for better generalization
    hsv_h=0.015,  # Hue augmentation
    hsv_s=0.7,    # Saturation augmentation
    hsv_v=0.4,    # Value augmentation
    degrees=10,    # Rotation augmentation
    translate=0.1, # Translation augmentation
    scale=0.5,     # Scale augmentation
    flipud=0.0,    # No vertical flip (cards have orientation)
    fliplr=0.5,    # Horizontal flip
    mosaic=1.0,    # Mosaic augmentation
    mixup=0.1,     # Mixup augmentation
    # Optimizer settings
    optimizer='AdamW',  # Better optimizer
    lr0=0.001,     # Initial learning rate
    lrf=0.01,      # Final learning rate fraction
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    # Better training
    box=7.5,       # Box loss gain
    cls=0.5,       # Class loss gain
    dfl=1.5,       # DFL loss gain
)

# Validate the model
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

# Export to ONNX for web deployment with specific settings
model = YOLO('runs/detect/card_detector_v2/weights/best.pt')

# Export with web-compatible settings
model.export(
    format='onnx',
    simplify=True,
    dynamic=False,
    imgsz=640,
    opset=12,  # Use older opset for better compatibility
)

print("\n✅ Model exported to ONNX!")
print("📥 Download the file: runs/detect/card_detector_v2/weights/best.onnx")

# Verify the export
import onnx
onnx_model = onnx.load('runs/detect/card_detector_v2/weights/best.onnx')
print(f"\nModel inputs: {[i.name for i in onnx_model.graph.input]}")
print(f"Model outputs: {[o.name for o in onnx_model.graph.output]}")
print("\nYou'll need this file for the web interface!")

# Download the file
from google.colab import files
files.download('runs/detect/card_detector_v2/weights/best.onnx')

# Test inference
import cv2
from google.colab.patches import cv2_imshow

# Test on a sample image
test_results = model.predict(source=f'{dataset.location}/test/images', save=True)
print(f"\n🎯 Test predictions saved to: runs/detect/predict")

INFO: pip is looking at multiple versions of tf2onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 127.8 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.9.23
    Uninstalling flatbuffers-25.9.23:
      Successfully uninstalled flatbuffers-25.9.23
  Attempting uninstall: opencv-python-headless
  


Extracting Dataset Version Zip to Playing-card-1 in yolov8:: 100%|██████████| 812/812 [00:00<00:00, 8289.10it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Playing-card-1/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ke

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


image 1/39 /content/Playing-card-1/test/images/014856104_jpg.rf.43686251142c3e649337dc13410c6ca3.jpg: 640x640 2 2Ss, 1 4D, 1 AH, 16.3ms
image 2/39 /content/Playing-card-1/test/images/081111127_jpg.rf.b679f18fd1a2af238142b06e089e42f2.jpg: 640x640 1 3D, 1 3H, 2 4Ds, 1 AS, 16.4ms
image 3/39 /content/Playing-card-1/test/images/197639949_jpg.rf.7e464c727a8496040c9b93b24a346ed5.jpg: 640x640 2 4Ds, 1 6H, 1 8S, 16.4ms
image 4/39 /content/Playing-card-1/test/images/290433499_jpg.rf.c3432eed505bca7f92febfcd817cb4be.jpg: 640x640 2 10Cs, 1 9S, 1 QC, 16.3ms
image 5/39 /content/Playing-card-1/test/images/297041922_jpg.rf.12b28dd99ee07efc43db3a14af245f54.jpg: 640x640 1 4S, 1 9S, 2 JDs, 16.3ms
image 6/39 /content/Playing-card-1/test/images/330166153_jpg.rf.c0810fd4285ec354bb5c11577472c8bc.jpg: 640x640 1 10H, 2 2Cs, 1 8D, 16.4ms
image 7/39 /content/Playing-card-1/test/images/348356833_jpg.rf.f6667a1498426c47fe03233a20550696.jpg: 640x640 2 10Ds, 1 3D, 1 QS, 16.4ms
image 8/39 /content/Playing-card-1/tes